In [15]:
import pandas as pd
import numpy as np
import yfinance as yf

def options_chain(symbol):

    tk = yf.Ticker(symbol)
    # Expiration dates
    exps = tk.options

    # Get options for each expiration
    options = pd.DataFrame()
    for e in exps:
        opt = tk.option_chain(e)
        opt = pd.DataFrame().append(opt.calls).append(opt.puts)
        opt['expirationDate'] = e
        options = options.append(opt, ignore_index=True)

    # Bizarre error in yfinance that gives the wrong expiration date
    # Add 1 day to get the correct expiration date
    options['expirationDate'] = pd.to_datetime(options['expirationDate']) + datetime.timedelta(days = 1)
    options['dte'] = (options['expirationDate'] - datetime.datetime.today()).dt.days / 365
    
    # Boolean column if the option is a CALL
    options['CALL'] = options['contractSymbol'].str[4:].apply(
        lambda x: "C" in x)
    
    options[['bid', 'ask', 'strike']] = options[['bid', 'ask', 'strike']].apply(pd.to_numeric)
    options['mark'] = (options['bid'] + options['ask']) / 2 # Calculate the midpoint of the bid-ask
    
    # Drop unnecessary and meaningless columns
    options = options.drop(columns = ['contractSize', 'currency', 'change', 'percentChange', 'lastTradeDate', 'lastPrice'])

    return options

In [24]:
symbol = 'AAPL'

tk = yf.Ticker(symbol)
print(tk.info)

JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [1]:
import requests
from bs4 import BeautifulSoup
headers = {
        "User-Agent": "Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:107.0) Gecko/20100101 Firefox/107.0"
    }
    # url = "https://finviz.com/quote.ashx?t=" + ticker.upper()
url = "https://finviz.com/screener.ashx?v=111&f=cap_mega,sh_opt_option&r=1"
req = requests.get(url, headers=headers)
print(req.text)
# get table from the req
soup = BeautifulSoup(req.text, 'html.parser')


<!DOCTYPE html>
<html lang="en" class=" dark">
<head>
<title>Stock Screener - Overview mega option </title>
<meta charset="UTF-8"><meta name="viewport" content="width=1024"><meta name="description" content="Stock screener for investors and traders, financial visualizations.">
<meta name="robots" content="noindex">

            <link rel="preload" href="/fonts/lato-v17-latin-ext_latin-regular.woff2" as="font" crossorigin>
            <link rel="preload" href="/fonts/lato-v17-latin-ext_latin-700.woff2" as="font" crossorigin>
            <link rel="preload" href="/fonts/lato-v17-latin-ext_latin-900.woff2" as="font" crossorigin>
            <link rel="preload" href="/fonts/inter-latin.woff2" as="font" crossorigin>
        
            <script>
                window.notificationsArray = [];
                window.renderScriptNotLoaded = function () {};
                window.handleScriptNotLoaded = function (element) {
                    window.notificationsArray.push(element);
          

In [27]:
from bs4 import BeautifulSoup

def extract_option_tables(soup):
    """
    Extracts call and put option tables from an HTML string.

    Args:
        soup: The HTML string containing the option tables.

    Returns:
        A tuple containing two lists of dictionaries, representing the call and put tables, respectively.
        Each dictionary represents a row in the table, with keys corresponding to the table headers.
    """

    options_section = soup.find('section', {'data-testid': 'options-list-table'})
    table_sections = options_section.find_all('div', class_='tableContainer yf-wurt5d')
    title_sections = options_section.find_all('div', class_='optionsHeader yf-wurt5d')

    tables, table_titles = [], []
    for table_section in table_sections:
        tables.append(table_section.find('table'))
    for title_section in title_sections:
        table_titles.append(title_section.find('h3').text.strip())

    call_table_data = []
    put_table_data = []

    for table, table_title in zip(tables, table_titles):

        headers = [th.text.strip() for th in table.find_all('th')]
        rows = table.find_all('tr')[1:]  # Skip header row

        table_data = []
        for row in rows:
            cells = row.find_all('td')
            row_data = {}
            for i, cell in enumerate(cells):
                row_data[headers[i]] = cell.text.strip()
            table_data.append(row_data)

        if table_title == 'Calls':
            call_table_data = table_data
        elif table_title == 'Puts':
            put_table_data = table_data

    return call_table_data, put_table_data


In [28]:
import requests
from bs4 import BeautifulSoup
headers = {
        "User-Agent": "Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:107.0) Gecko/20100101 Firefox/107.0"
    }
url = "https://finance.yahoo.com/quote/NVDA/options/?date=1744848000"
req = requests.get(url, headers=headers)
# print(req.text)
# get table from the req
soup = BeautifulSoup(req.text, 'html.parser')

call_data, put_data = extract_option_tables(soup)
print("Calls:", call_data)
print("Puts:", put_data)

Calls: [{'Contract Name': 'NVDA250417C00005000', 'Last Trade Date (EDT)': '3/14/2025  11:27 AM', 'Strike': '5', 'Last Price': '116.11', 'Bid': '116.20', 'Ask': '117.00', 'Change': '4.31', '% Change': '3.86%', 'Volume': '1', 'Open Interest': '13,103', 'Implied Volatility': '561.72%'}, {'Contract Name': 'NVDA250417C00010000', 'Last Trade Date (EDT)': '3/11/2025  12:49 PM', 'Strike': '10', 'Last Price': '98.97', 'Bid': '111.25', 'Ask': '112.00', 'Change': '0.00', '% Change': '0.00%', 'Volume': '82', 'Open Interest': '127', 'Implied Volatility': '424.61%'}, {'Contract Name': 'NVDA250417C00015000', 'Last Trade Date (EDT)': '3/10/2025  3:00 PM', 'Strike': '15', 'Last Price': '91.30', 'Bid': '104.40', 'Ask': '107.05', 'Change': '0.00', '% Change': '0.00%', 'Volume': '21', 'Open Interest': '387', 'Implied Volatility': '359.77%'}, {'Contract Name': 'NVDA250417C00020000', 'Last Trade Date (EDT)': '3/11/2025  12:22 PM', 'Strike': '20', 'Last Price': '88.98', 'Bid': '101.20', 'Ask': '102.05', 'Cha

In [4]:
# Find the main table containing stock data
tables = soup.find_all('table')

for table in tables:
    class_attr = table.get('class')
    class_name = ' '.join(class_attr) if class_attr else "No class"
    print(f"Table Class: {class_name}")
    if class_name.__contains__("styled-table-new"):
        print(f"Table HTML:\n{table}\n{'-' * 40}")
    # print(f"Table HTML:\n{table}\n{'-' * 40}")  # Separator

Table Class: header
Table Class: header-container
Table Class: w-full
Table Class: navbar
Table Class: header-container
Table Class: fv-container is-tight
Table Class: screener-groups_table-filter filter-table-filters
Table Class: No class
Table Class: screener-groups_settings-table is-dimmer-text
Table Class: screener_custom-input-table is-modal
Table Class: screener_custom-input-table is-modal
Table Class: screener_custom-input-table is-modal
Table Class: screener_custom-input-table is-modal
Table Class: screener_custom-input-table is-modal
Table Class: screener_custom-input-table is-modal
Table Class: screener_custom-input-table is-modal
Table Class: screener_custom-input-table is-modal
Table Class: screener_custom-input-table is-modal
Table Class: No class
Table Class: screener-view-table screener-view-switch
Table Class: No class
Table Class: styled-table-new is-rounded is-tabular-nums w-full screener_table
Table HTML:
<table class="styled-table-new is-rounded is-tabular-nums w-fu

In [7]:
pagination_td = soup.find('td', id='screener_pagination')

if pagination_td:
    links = pagination_td.find_all('a')
    unselected_links = [link for link in links if 'is-selected' not in link.get('class', [])]

    for link in unselected_links:
        if link.text == '' or link.text == 'None':
            print(f"Next Page URL: {link.get('href')}")
            break
        print(f"Link Text: {link.text}")
        print(f"Link HREF: {link.get('href')}")
        print("-" * 20)
else:
    print("Pagination element not found.")

Link Text: 2
Link HREF: screener.ashx?v=111&f=cap_mega,sh_opt_option&r=21
--------------------
Link Text: 3
Link HREF: screener.ashx?v=111&f=cap_mega,sh_opt_option&r=41
--------------------
Link Text: 


Link HREF: screener.ashx?v=111&f=cap_mega,sh_opt_option&r=21
--------------------


<>:8: SyntaxWarning: "is" with a literal. Did you mean "=="?
<>:8: SyntaxWarning: "is" with a literal. Did you mean "=="?
/var/folders/t1/2w1wkbqj729by6cwg2dd4f_w0000gn/T/ipykernel_91667/3920974284.py:8: SyntaxWarning: "is" with a literal. Did you mean "=="?
  if link.text == '' or link.text is 'None':


In [14]:
pagination_td = soup.find('td', id='screener_pagination')

if pagination_td:
    next_link = None
    links = pagination_td.find_all('a')
    for link in links:
        if link.get('class').__contains__('is-next'):
            next_link = link
            break  # Stop searching once "next" is found

    if next_link:
        print(f"Next Link HREF: {next_link.get('href')}")
    else:
        print("No 'next' link found.")
else:
    print("Pagination element not found.")

Next Link Text: 


Next Link HREF: screener.ashx?v=111&f=cap_mega,sh_opt_option&r=21


In [3]:
print(table)

None


In [4]:
print(soup.prettify())

�
<!DOCTYPE html>
hED5�@2�}����S��{ScRc�"��M�L9��h���dj�O��&amp; ��I\5�þ�e���Z�-�E,� ���=𿾀l�Hr�d	����3�����/9�˜x&lt;)�ff�g���YoJE�:.)� ��}�����p8\����?�����.�v:��y�B�톽�n\+.�j)#(k�5{v�28��k��=�K�(�@ �=���Z�)�j@f��u[6�Xt9�x?/"i�Q�
&lt;��8J&amp;��#UW�b��w�]"Y��
<c��s�^5ې� tf\�k8��'��[�ʾb�e��;i�="" '�& ���k����1��n�*��="" 4�:�������a�z�="" 9mvl��~p��="" `9�똇="" b�l="0\��" c��1����="" e���!�jigg�:��<�q����ew��)��*!@mf*%="" i���'�d�'s���ԇ.��="" j�w�z?~��{0(2�="" o8�,�1��yn1�="" r٪�)�s��t����f�|�#^��q���_��="" t*�v�x'g�r�f�b�9�c�k�m{vg����t4*[="" yo="" ԍ7o�="" ׮+t ����,�wϋ��������2�v;wvt�yv��h="" ��]?)��(cv`�*�?�="" �_�5�h������\�4)��"��✼t��n;k�����tr�="" �k�a���="" �܁oؖ��6#��8np�="" ���2���}�-w��xvbn^(s�^e�l�i�g@�gm="" ��(kg��y���y5��.o��#k�h��*="" ��[�;۠����-jgn���b�p�tx-$�6ܳ��z�3t�6��$�6�n�?5j-:;bܺ@`m����,��fx�gxqd�����dc�="">
 /��}�b=5p��8G�pL��{C[$�&gt;g(c��-j_��e*Q���@��x\A���.���Ni_�j�Z
��������,Ϣ�DK

In [7]:
import pandas as pd
from bs4 import BeautifulSoup

def get_ticker(ticker):
    # ---------- Pulling the data from chosen stock ticker ----------#

    headers = {
        "User-Agent": "Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:107.0) Gecko/20100101 Firefox/107.0"
    }
    url = "https://finviz.com/quote.ashx?t=" + ticker.upper()
    req = requests.get(url, headers=headers)
    table = pd.read_html(req.text, attrs={"class": "snapshot-table2"})
    df = table[0]
    return (df[1][3], df[1][2])


print(get_ticker("AAPL"))

ImportError: html5lib not found, please install it

In [ ]:



if not table:
    print("No table found on the page")


# Get all the rows (skip header row)
rows = table.select('tr[valign="top"]')

In [1]:
import yfinance as yf
dat = yf.Ticker("MSFT")

In [2]:
ticker = "MSFT"

In [12]:
batch_tickers = ['MSFT']
batch_data = yf.download(
                tickers=batch_tickers,
                period="5d",
                interval="1d",
                group_by="ticker",
                auto_adjust=True,
                prepost=True,
                threads=True,
                proxy=None
            )

[*********************100%***********************]  1 of 1 completed

1 Failed download:
- MSFT: No data found for this date range, symbol may be delisted


In [17]:
batch_tickers = ["SPY", "AAPL", "MSFT"]

batch_data = yf.download(
    tickers=batch_tickers,
    period="5d",
    interval="1d",  # Try changing to a higher interval
    group_by="ticker",
    auto_adjust=True,
    prepost=True,  # Include pre-market and after-hours data
    threads=True,
    proxy=None
)

[*********************100%***********************]  3 of 3 completed

3 Failed downloads:
- MSFT: No data found for this date range, symbol may be delisted
- SPY: No data found for this date range, symbol may be delisted
- AAPL: No data found for this date range, symbol may be delisted


In [18]:
data = yf.download("AAPL", period="5d", interval="1d")
print(data)

[*********************100%***********************]  1 of 1 completed

1 Failed download:
- AAPL: No data found for this date range, symbol may be delisted
Empty DataFrame
Columns: [Open, High, Low, Close, Adj Close, Volume]
Index: []


In [19]:
import yfinance as yf

ticker = yf.Ticker("AAPL")
print(ticker.history(period="5d"))


- AAPL: No data found for this date range, symbol may be delisted
Empty DataFrame
Columns: [Open, High, Low, Close, Adj Close, Volume]
Index: []


In [4]:
stock = yf.Ticker(ticker)
                    # Attempt to fetch options expiration dates
                    # If options are available, this will return a list of dates
options = stock.options

# if options and len(options) > 0:
#     tickers_with_options.append(ticker)
#     logger.info(f"Ticker {ticker} has options")
# else:
#     logger.info(f"Ticker {ticker} does not have options")

JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [20]:
dat.history(period='1mo')

- MSFT: No data found for this date range, symbol may be delisted


,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,


In [6]:
dat.info

JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [21]:
from yahoo_fin import stock_info as si

ticker = 'AAPL'
# Get current price
current_price = si.get_live_price(ticker)

Warning - Certain functionality 
             requires requests_html, which is not installed.
             
             Install using: 
             pip install requests_html
             
             After installation, you may have to restart your Python session.


JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [ ]:
# Retrieve all options chains for the ticker
# yahoo_fin provides a list of expiration dates:
expiration_dates = si.get_expiration_dates(ticker)

# Let's assume the nearest Friday is the first date in the list:
next_friday = expiration_dates[0]

# Get the option chain for that date
option_chain = si.get_options_chain(ticker, next_friday)
calls = option_chain['calls']
puts = option_chain['puts']

# Filter for ±20% around the current price
lower_strike = 0.8 * current_price
upper_strike = 1.2 * current_price

calls_filtered = calls[(calls['Strike'] >= lower_strike) & (calls['Strike'] <= upper_strike)]
puts_filtered  = puts[(puts['Strike']  >= lower_strike) & (puts['Strike']  <= upper_strike)]

return calls_filtered, puts_filtered